The following code generates 30 random JSON objects and sends each object to the Lambda function `LBC-createPotholeComplaint` to populate the `complaints` table in the DynamoDB database. The code uses the `boto3` library to interact with AWS services and the `json` library to work with JSON objects. The code generates random values for the following fields in the JSON object:
- `complaintId`: A unique identifier for the complaint. It is generated using the `generate_complaint_id` function. The complaint ID has the format `COMP-` followed by a random alphanumeric string of length 10
- `userId`: A unique identifier for the user who submitted the complaint. It is generated using the `generate_user_id` function. The user ID has the format `USER-` followed by a random alphanumeric string of length 8
- `streetName`: A random string of uppercase letters and digits of length between 5 and 15
- `nearbyLandmark`: A random string of uppercase letters and digits of length between 10 and 30
- `longitude`: A random float value between -180 and 180
- `latitude`: A random float value between -90 and 90
- `comments`: A random string of letters and digits of length between 10 and 30
- `addedOn`: The current date and time in ISO format
- `status`: A random choice between `IN_PROGRESS` and `COMPLETED`
- `base64Images`: A list of random base64 strings representing images
- The code then sends each JSON object to the Lambda function `LBC-createPotholeComplaint` using the `invoke` method of the Lambda client. The response from the Lambda function is printed, which includes the status code, function error, and the payload returned by the Lambda function.
- The code is useful for populating the `complaints` table with test data for testing the functionality of the Lambda function that creates pothole complaints in the system.
- The code can be run in a Jupyter notebook or any Python environment with the necessary libraries installed.
- The `boto3` library must be installed to interact with AWS services.
- The code can be modified to generate different test data or to send the data to a different Lambda function.
- The code assumes that the Lambda function `LBC-createPotholeComplaint` is deployed and accessible in the AWS account.
- The code can be run multiple times to generate and send more test data to the Lambda function.
- The code can be used to test the scalability and performance of the Lambda function by sending a large number of requests in a short period.
- The code can be used to test the error handling and response handling of the Lambda function by analyzing the response from the Lambda function.

In [ ]:
import json
import random
import string
from datetime import datetime, timezone
import base64
import boto3

# Constants
COMPLAINT_ID_LENGTH = 10
USER_ID_LENGTH = 8
LAMBDA_FUNCTION_NAME = 'LBC-createPotholeComplaint' #Change this to the name of your Lambda function

# AWS Lambda client
lambda_client = boto3.client('lambda')

# Helper functions
def generate_complaint_id():
    """
    Function to generate a complaint ID.
    Returns a string of the form "COMP-" followed by a random alphanumeric string of length COMPLAINT_ID_LENGTH.
    """
    return "COMP-" + ''.join(random.choices(string.ascii_letters + string.digits, k=COMPLAINT_ID_LENGTH))

def generate_user_id():
    """
   Function to generate a user ID.
   Returns a string of the form "USER-" followed by a random alphanumeric string of length USER_ID_LENGTH.
   """
    return "USER-" + ''.join(random.choices(string.ascii_letters + string.digits, k=USER_ID_LENGTH))

def get_iso_datetime_string():
    """
    Function to get the current date and time in ISO format.
    Returns a string representing the current date and time in ISO format.
    """
    now = datetime.now(tz=timezone.utc)
    iso_datetime = now.isoformat(timespec='milliseconds')
    return iso_datetime

def generate_random_latitude():
    """
    Function to generate a random latitude.
    Returns a float representing a random latitude between -90 and 90.
    """
    return random.uniform(-90, 90)

def generate_random_longitude():
    """
    Function to generate a random longitude.
    Returns a float representing a random longitude between -180 and 180.
    """
    return random.uniform(-180, 180)

def generate_random_status():
    """
    Function to generate a random longitude.
    Returns a float representing a random longitude between -180 and 180.
    """
    return random.choice(["ACTIVE", "COMPLETED"])

def generate_base64_image():
    """
    Function to generate a random base64 string.
    Returns a string representing a random base64 string.
    """
    # Generate a random base64 string
    random_bytes = bytearray(random.getrandbits(8) for _ in range(random.randint(100, 1000)))
    return base64.b64encode(random_bytes).decode('utf-8')

# Generate 30 JSON objects
json_objects = []
for _ in range(30):
    num_images = random.randint(1, 5)  # Random number of images between 1 and 5
    base64_images = [generate_base64_image() for _ in range(num_images)]

    obj = {
        # "complaintId": generate_complaint_id(),
        # "userId": generate_user_id(),
        "streetName": "".join(random.choices(string.ascii_uppercase + string.digits, k=random.randint(5, 15))),
        "nearbyLandmark": "".join(random.choices(string.ascii_uppercase + string.digits, k=random.randint(10, 30))),
        "longitude": generate_random_longitude(),
        "latitude": generate_random_latitude(),
        "comments": "".join(random.choices(string.ascii_letters + string.digits, k=random.randint(10, 30))),
        # "addedOn": get_iso_datetime_string(),
        "status": generate_random_status(),
        "base64Images": base64_images
    }
    json_objects.append(obj)

# Send each JSON object to the Lambda function and get the response
for obj in json_objects:
    print(json.dumps(obj, indent=2))
    response = lambda_client.invoke(
        FunctionName=LAMBDA_FUNCTION_NAME,
        InvocationType='RequestResponse',  # Change this
        Payload=json.dumps(obj)
    )
    status_code = response.get('StatusCode', 'No StatusCode')
    function_error = response.get('FunctionError', 'No FunctionError')

    # Read the returned message
    payload = response['Payload'].read()
    payload_decoded = payload.decode('utf-8')
    payload_json = json.loads(payload_decoded)

    print(f"Response from Lambda: {status_code} - {function_error} - {payload_json}")